In [78]:
import os
import sys

def insert (source_str, insert_str, pos):
    return source_str[:pos]+insert_str+source_str[pos:]

filename = "test"
content = open(filename + ".csv").readlines()
outfile = filename + ".txt"

PID = {
"0001" : "OUT    ",
"1001" : "IN     ",
"0101" : "SOF    ",
"1101" : "SETUP  ",
"0011" : "DATA0  ",
"1011" : "DATA1  ",
"0111" : "DATA2  ",
"1111" : "MDATA  ",
"0010" : "ACK    ",
"1010" : "NAK    ",
"1110" : "STALL  ",
"0110" : "NYET   ",
"1100" : "ERR    ",
"1000" : "SPLIT  ",
"0100" : "PING   "}

result = []
i = 0
while i < len(content):
    j = content[i].split(",")
    try:
        assert ("SOP" in j[2])
    except AssertionError:
        if "Reset" in j[2] or "USB signalling" in j[2]:
            i += 1
            j = content[i].split(",")
            continue
        else:
            print("Error at " + i)
            break
    time = j[1]
    s = ""
    g = ""
    i+=1
    j = content[i].split(",")
    while "EOP" not in j[2]:
        if "Stuff" in j[2]:
            i+=1
            j = content[i].split(",")
            continue
        g+=j[2].replace("\n", "")
        if len(g) == 4:
            s+=hex(int(g, 2))[2:]
            g = ""
        i+=1
        j = content[i].split(",")
    assert(s[0:2] == "01")
    pidtype = PID[(bin(int(s[2], 16))[2:].zfill(4))[::-1]]
    s = s[4:]
    if "IN" in pidtype or "OUT" in pidtype or "SETUP" in pidtype:
        temp = int(s, 16)
        s = "addr: " + hex(int(bin((temp >> 9) & 0x7f)[2:].zfill(7)[::-1], 2)) + ", endpoint: " + hex(int(bin((temp >> 5) & 0x0f)[2:].zfill(4)[::-1], 2))
    if "DATA" in pidtype:
        t = s[:-4]
        s = ""
        for a, b in zip(t[::2], t[1::2]):
            s += hex(int((bin(int(a+b, 16))[2:].zfill(8))[::-1], 2))[2:].zfill(2)
        if len(s) == 128:
            g = " NS_KEY: "
            g += s[6:12] + " "
            g += s[15] + s[12] + s[13] + " "
            g += s[16:18] + s[14] + " "
            g += s[21] + s[18] + s[19] + " "
            g += s[22:24] + s[20] + " "
            s = insert(s, " ", 24)
            s += g
    if "SOF" not in pidtype:
        result.append((pidtype + s + " ", time))
    i+=1
    
if os.path.exists(outfile):
    os.remove(outfile)
f = open(outfile, "x")

for i in result:
    if"IN" in i[0] or "OUT" in i[0] or "SETUP" in i[0]:
        f.write(i[1].ljust(9) + ", ")
    f.write(i[0])
    if "IN" not in i[0] and "OUT" not in i[0] and "DATA" not in i[0] and "SETUP" not in i[0]:
        f.write("\n")
f.close()


In [15]:
content = open(outfile).readlines()
os.remove(outfile)
f = open(outfile, "x")
for i in content:
    if "NAK" not in i:
        f.write(i)
f.close()

In [60]:
somestr = "300091 008000 808080808080 00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"
somestr = somestr.replace(" ", "")
i = 0
print(len(somestr)/2)
while i < len(somestr):
    print("0x", end='')
    print(somestr[i], end='')
    i += 1
    print(somestr[i], end='')
    i += 1
    print(",",end='')


64.0
0x30,0x00,0x91,0x00,0x80,0x00,0x80,0x80,0x80,0x80,0x80,0x80,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,